In [1]:
import platform,os, sys
os_name = platform.system()

In [2]:
import findspark

In [3]:
if os_name == "Windows":
    findspark.init()
    img_dir = "D:/Learn/GitRepos/Spark-The-Definitive-Guide/data/deep-learning-images"
    temp_loc = "D:/temp/"
    master = "local[*]"
else:
    spark_home = "/usr/hdp/current/spark2-client"
    findspark.init(spark_home)
    hdfs_home = "/user/" + os.getenv("HOME").split("/")[2]
    img_dir = hdfs_home+"/dataSets/spark-guide/deep-learning-images/"
    temp_loc = os.getenv("HOME") + "/temp/"
    master = "yarn"
    #sys.path.append(os.path.expanduser('hdfs:///user/kranthidr/pythonpath/sparkdl.zip'))
    sys.path.append(os.path.expanduser('~/pythonpath/sparkdl.zip'))
    sys.path.append(os.path.expanduser('~/pythonpath/tensorframes.zip'))
    sys.path.append(os.path.expanduser('~/pythonpath/tensorflow.zip'))
    sys.path.append(os.path.expanduser("~/.local/lib/python2.7/site-packages"))

In [4]:
sys.path

['/usr/hdp/current/spark2-client/python',
 '/usr/hdp/current/spark2-client/python/lib/py4j-0.10.6-src.zip',
 '',
 '/usr/lib/python2.7/site-packages/avro-1.7.7-py2.7.egg',
 '/usr/lib64/python27.zip',
 '/usr/lib64/python2.7',
 '/usr/lib64/python2.7/plat-linux2',
 '/usr/lib64/python2.7/lib-tk',
 '/usr/lib64/python2.7/lib-old',
 '/usr/lib64/python2.7/lib-dynload',
 '/home/kranthidr/.local/lib/python2.7/site-packages',
 '/usr/lib64/python2.7/site-packages',
 '/usr/lib64/python2.7/site-packages/gtk-2.0',
 '/usr/lib/python2.7/site-packages',
 '/usr/lib/python2.7/site-packages/IPython/extensions',
 '/home/kranthidr/.ipython',
 '/home/kranthidr/pythonpath/sparkdl.zip',
 '/home/kranthidr/pythonpath/tensorframes.zip',
 '/home/kranthidr/pythonpath/tensorflow.zip',
 '/home/kranthidr/.local/lib/python2.7/site-packages']

In [5]:
findspark.find()

'/usr/hdp/current/spark2-client'

In [6]:
img_dir

'/user/kranthidr/dataSets/spark-guide/deep-learning-images/'

In [7]:
temp_loc

'/home/kranthidr/temp/'

In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]")\
.config("spark.driver.memory", "2g")\
.config("spark.jars.packages","databricks:spark-deep-learning:1.2.0-spark2.3-s_2.11,databricks:tensorframes:0.5.0-s_2.11")\
.appName("ch31_DeepLearningPopModels").getOrCreate()

#.config("spark.jars.packages","databricks:spark-deep-learning:1.2.0-spark2.3-s_2.11,databricks:tensorframes:0.5.0-s_2.11")\
# .config("spark.driver.memory", "8g")\
# .config("spark.executor.memory", "3g")\
# .config("spark.executor.cores", "4")\
# .config("spark.executor.instances", "10")\


#!/usr/bin/env bash

# This file is sourced when running various Spark programs.
# Copy it as spark-env.sh and edit that to configure Spark for your site.

# Options read in YARN client mode
#SPARK_EXECUTOR_INSTANCES="2" #Number of workers to start (Default: 2)
#SPARK_EXECUTOR_CORES="1" #Number of cores for the workers (Default: 1).
#SPARK_EXECUTOR_MEMORY="1G" #Memory per Worker (e.g. 1000M, 2G) (Default: 1G)
#SPARK_DRIVER_MEMORY="512M" #Memory for Master (e.g. 1000M, 2G) (Default: 512 Mb)
#SPARK_YARN_APP_NAME="spark" #The name of your application (Default: Spark)
#SPARK_YARN_QUEUE="default" #The hadoop queue to use for allocation requests (Default: default)
#SPARK_YARN_DIST_FILES="" #Comma separated list of files to be distributed with the job.
#SPARK_YARN_DIST_ARCHIVES="" #Comma separated list of archives to be distributed with the job.

In [9]:
sc = spark.sparkContext

In [10]:
for x in sc._conf.getAll():
    if "/proxy/" in x[1]:
        print(x[1])

In [11]:
spark

In [12]:
from sparkdl.image import imageIO
from sparkdl import DeepImagePredictor

from pyspark.ml.image import ImageSchema

/home/kranthidr/.local/lib/python2.7/site-packages/pandas/core/nanops.py:39: UserWarning: The installed version of bottleneck 0.7.0 is not supported in pandas and will be not be used
The minimum supported version is 1.0.0

  ver=ver, min_ver=_MIN_BOTTLENECK_VERSION), UserWarning)
Using TensorFlow backend.


In [13]:
from pyspark.sql.functions import lit
from pyspark.sql.functions import expr
from pyspark.sql.types import DoubleType

In [14]:
tulips_df = ImageSchema.readImages(img_dir + "/tulips").withColumn("label", lit(1))
tulips_df.count()

# Signature: ImageSchema.readImages(path, recursive=False, 
#                                   numPartitions=-1, dropImageFailures=False, 
#                                   sampleRatio=1.0, seed=0)
# Docstring:
# Reads the directory of images from the local or remote source.

# .. note:: If multiple jobs are run in parallel with different sampleRatio or recursive flag,
#     there may be a race condition where one job overwrites the hadoop configs of another.

# .. note:: If sample ratio is less than 1, sampling uses a PathFilter that is efficient but
#     potentially non-deterministic.

# :param str path: Path to the image directory.
# :param bool recursive: Recursive search flag.
# :param int numPartitions: Number of DataFrame partitions.
# :param bool dropImageFailures: Drop the files that are not valid images.
# :param float sampleRatio: Fraction of the images loaded.
# :param int seed: Random number seed.
# :return: a :class:`DataFrame` with a single column of "images",
#        see ImageSchema for details.

30

In [15]:
# In[16]:
daisy_df = imageIO.readImagesWithCustomFn(img_dir + "/daisy", decode_f=imageIO.PIL_decode).withColumn("label", lit(0))
daisy_df.count()

# Signature: imageIO.readImagesWithCustomFn(path, decode_f, numPartition=None)
# Docstring:
# Read a directory of images (or a single image) into a DataFrame using a custom library to
# decode the images.

# :param path: str, file path.
# :param decode_f: function to decode the raw bytes into an array compatible with one of the
#     supported OpenCv modes. see @imageIO.PIL_decode for an example.

30

In [16]:
# In[ ]:
tulips_train, tulips_test = tulips_df.randomSplit([0.6, 0.4])
daisy_train, daisy_test = daisy_df.randomSplit([0.6, 0.4])

In [17]:
# In[ ]:
train_df = tulips_train.unionAll(daisy_train).cache()
test_df = tulips_test.unionAll(daisy_test).cache()

In [18]:
train_df.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = false)
 |    |-- width: integer (nullable = false)
 |    |-- nChannels: integer (nullable = false)
 |    |-- mode: integer (nullable = false)
 |    |-- data: binary (nullable = false)
 |-- label: integer (nullable = false)



In [19]:
train_df.show(10)

+--------------------+-----+
|               image|label|
+--------------------+-----+
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
+--------------------+-----+
only showing top 10 rows



In [20]:
test_df.show(10)

+--------------------+-----+
|               image|label|
+--------------------+-----+
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
+--------------------+-----+
only showing top 10 rows



In [21]:
image_df = train_df.unionAll(test_df)

In [22]:
image_df.show(10)

+--------------------+-----+
|               image|label|
+--------------------+-----+
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
|[hdfs://nn01.itve...|    1|
+--------------------+-----+
only showing top 10 rows



In [23]:
import keras.applications
from keras.applications.inception_v3 import InceptionV3

In [24]:
predictor = DeepImagePredictor(
  inputCol="image",
  outputCol="predicted_labels",
  modelName="InceptionV3",
  decodePredictions=True,
  topK=10)

# Init signature: DeepImagePredictor(inputCol=None, outputCol=None, 
#                                    modelName=None, decodePredictions=False, topK=5)
# Docstring:     
# Applies the model specified by its popular name to the image column in DataFrame.
# The input image column should be 3-channel SpImage.
# The output is a MLlib Vector.

In [25]:
predictions_base_model_df = predictor.transform(test_df).cache()

INFO:tensorflow:Froze 378 variables.
Converted 378 variables to const ops.
INFO:tensorflow:Froze 0 variables.
Converted 0 variables to const ops.


In [26]:
#%env KERAS_BACKEND=tensorflow

In [27]:
import os
print(os.getenv("KERAS_BACKEND"))

None


In [28]:
predictions_base_model_df.show(5)

+--------------------+-----+--------------------+
|               image|label|    predicted_labels|
+--------------------+-----+--------------------+
|[hdfs://nn01.itve...|    1|[[n01855672, goos...|
|[hdfs://nn01.itve...|    1|[[n03457902, gree...|
|[hdfs://nn01.itve...|    1|[[n03930313, pick...|
|[hdfs://nn01.itve...|    1|[[n03590841, jack...|
|[hdfs://nn01.itve...|    1|[[n03944341, pinw...|
+--------------------+-----+--------------------+
only showing top 5 rows



In [29]:
k = "hdfs://nn01.itversity.com:8020/user/kranthidr/dataSets/spark-guide/deep-learning-images/tulips/3498663243_42b39b4185_m.jpg"

In [30]:
def getName(filname):
    return filname.split("/")[-1]

In [31]:
getName(k)

'3498663243_42b39b4185_m.jpg'

In [32]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

get_name = udf(getName, StringType())

In [33]:
from pyspark.sql.functions import explode
predictions_base_model_df.select("label", get_name("image.origin"),explode("predicted_labels")).show(30,False)

+-----+---------------------------+--------------------------------------------+
|label|getName(image.origin)      |col                                         |
+-----+---------------------------+--------------------------------------------+
|1    |3501996215_1c6d1a3386_n.jpg|[n01855672, goose, 0.20516284]              |
|1    |3501996215_1c6d1a3386_n.jpg|[n02007558, flamingo, 0.15303424]           |
|1    |3501996215_1c6d1a3386_n.jpg|[n01860187, black_swan, 0.1488998]          |
|1    |3501996215_1c6d1a3386_n.jpg|[n09332890, lakeside, 0.096883185]          |
|1    |3501996215_1c6d1a3386_n.jpg|[n02009912, American_egret, 0.048992112]    |
|1    |3501996215_1c6d1a3386_n.jpg|[n02051845, pelican, 0.015660044]           |
|1    |3501996215_1c6d1a3386_n.jpg|[n02012849, crane, 0.008230081]             |
|1    |3501996215_1c6d1a3386_n.jpg|[n03457902, greenhouse, 0.008111391]        |
|1    |3501996215_1c6d1a3386_n.jpg|[n02018207, American_coot, 0.006786133]     |
|1    |3501996215_1c6d1a3386